In [231]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="white")
import os
import plotly.express as px
from plotly.subplots import make_subplots
from plotly import graph_objects as go

In [232]:
cols={0:"time_string",1:"time_taken_lec",2:"time_taken_safety_controller",3:"total_time",4:"moet",5:"mean_et",6:"avg_iterations"}
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0","1.5"]
obstacles = ["no_obstacles","with_obstacles"]
file_str = "{}/benchmark_experiments_{}_track_porto_{}.csv"
dfs = []
for alg in algorithms:
    for speed in speeds:
        for obs_presence in obstacles:
            filepath = file_str.format(obs_presence,alg,speed)
            df = df = pd.read_csv(filepath,header=None).rename(columns=cols).tail(30)
            df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
            df=df.drop(columns=["time_taken_lec","total_time","time_taken_safety_controller"])
            df["sc_usage"] = 1 - df["ml_usage"]
            df["ml_usage"]=df["ml_usage"]*100
            df["sc_usage"]=df["sc_usage"]*100
            df["ml_controller"] = alg.replace("_rtreach","")
            df["speed"] = speed
            df["obstacle_presence"] = obs_presence
            dfs.append(df)

In [233]:
def replace_e2e(row):
    if(row=="e2e_image"):
        return "E2E (Camera)"
    elif(row=="e2e"):
        return "E2E (Lidar)"
    else:
        return row.upper()
def replace_obs(row):
    if(row=="no_obstacles"):
        return "No" 
    else: 
        return "Yes"


all_dfs = pd.concat(dfs)
all_dfs["ml_controller"]=all_dfs["ml_controller"].apply(replace_e2e)
all_dfs["obstacle_presence"] = all_dfs["obstacle_presence"].apply(replace_obs)

In [234]:
res = round(all_dfs.groupby(["ml_controller","speed","obstacle_presence"]).agg(["mean","std"]),3)
res

moet        mean_et         \
                                        mean    std    mean    std   
ml_controller speed obstacle_presence                                
ARS           0.5   No                 0.074  0.012   0.027  0.002   
                    Yes                0.061  0.001   0.026  0.001   
              1.0   No                 0.061  0.011   0.022  0.004   
                    Yes                0.062  0.008   0.024  0.004   
              1.5   No                 0.059  0.001   0.020  0.003   
                    Yes                0.061  0.005   0.022  0.003   
E2E (Camera)  0.5   No                 0.057  0.004   0.034  0.001   
                    Yes                0.060  0.005   0.034  0.001   
              1.0   No                 0.058  0.002   0.033  0.001   
                    Yes                0.062  0.013   0.033  0.001   
              1.5   No                 0.061  0.013   0.030  0.003   
                    Yes                0.061  0.002   0.032  0.001   
E2E (Lidar)   0.5   No                 0.053  0.005   0.036  0.001   
                    Yes                0.059  0.002   0.034  0.001   
              1.0   No                 0.058  0.017   0.035  0.005   
                    Yes                0.060  0.006   0.034  0.003   
              1.5   No                 0.058  0.009   0.036  0.006   
                    Yes                0.059  0.003   0.032  0.007   
SAC           0.5   No                 0.086  0.005   0.028  0.002   
                    Yes                0.061  0.004   0.027  0.001   
              1.0   No                 0.058  0.001   0.027  0.006   
                    Yes                0.066  0.037   0.024  0.003   
              1.5   No                 0.061  0.008   0.026  0.004   
                    Yes                0.060  0.002   0.026  0.004   

                                      avg_iterations        ml_usage          \
                                                mean    std     mean     std   
ml_controller speed obstacle_presence                                          
ARS           0.5   No                        10.263  0.230   58.861   2.580   
                    Yes                       10.011  0.368   55.263   2.951   
              1.0   No                        11.627  1.628   32.920   7.150   
                    Yes                       11.385  1.656   26.392   3.813   
              1.5   No                        12.110  1.589   16.641   2.536   
                    Yes                       11.832  1.539   14.111   2.042   
E2E (Camera)  0.5   No                         6.011  0.015   94.295   0.357   
                    Yes                        6.017  0.031   84.752   2.683   
              1.0   No                         6.183  0.167   73.246  12.113   
                    Yes                        6.772  0.703   49.731   8.185   
              1.5   No                         7.688  1.457   40.098  13.355   
                    Yes                        6.970  0.761   36.038  10.781   
E2E (Lidar)   0.5   No                         6.004  0.019  100.000   0.000   
                    Yes                        6.016  0.013   92.457   1.630   
              1.0   No                         6.500  2.141   89.399  18.342   
                    Yes                        6.979  1.180   52.298   6.353   
              1.5   No                         6.681  2.130   26.614  17.679   
                    Yes                        7.269  2.905   25.661  10.147   
SAC           0.5   No                         9.601  0.257   55.621   6.373   
                    Yes                        9.508  0.162   53.294   3.660   
              1.0   No                         9.009  2.522   16.984  10.246   
                    Yes                       10.860  1.604   18.966   3.152   
              1.5   No                         9.699  1.890   10.274   3.585   
                    Yes                        9.695  1.721    9.245 

In [235]:
cols={0:"time_string",1:"time_taken_lec",2:"time_taken_safety_controller",3:"total_time",4:"moet",5:"mean_et",6:"avg_iterations"}
df = pd.read_csv(filepath,header=None).rename(columns=cols)
df["ML_usage"] = df["time_taken_lec"]/df["total_time"]
df=df.drop(columns=["time_taken_lec","total_time","time_taken_safety_controller"])
df.agg(["mean","std"])

,moet,mean_et,avg_iterations,ML_usage
mean,0.061328,0.022118,11.832048,0.141112
std,0.005247,0.003398,1.538529,0.020422


In [236]:
all_dfs = pd.concat(dfs)
all_dfs["ml_controller"]=all_dfs["ml_controller"].apply(replace_e2e)
all_dfs["obstacle_presence"] = all_dfs["obstacle_presence"].apply(replace_obs)
all_dfs[all_dfs.obstacle_presence=="Yes"]
all_dfs['moet'] = all_dfs['moet'] *1000
all_dfs['mean_et'] = all_dfs['mean_et'] * 1000

In [237]:
with_obs = all_dfs[all_dfs.obstacle_presence=="Yes"]
no_obs = all_dfs[all_dfs.obstacle_presence=="No"]
#with_obs[with_obs.ml_controller=="E2e"].groupby("speed").agg(["mean","std"])

In [238]:
with_obs[list(with_obs.columns[with_obs.columns.isin(['moet','mean_et','avg_iterations','speed'])])].groupby("speed").agg(["mean","std"])

moet               mean_et           avg_iterations          
            mean        std       mean       std           mean       std
speed                                                                    
0.5    60.343092   3.609197  30.270057  4.097035       7.887978  1.898403
1.0    62.805250  19.935807  28.695691  5.710042       8.998741  2.518410
1.5    60.298433   3.364057  28.170907  6.116465       8.941194  2.726182

In [239]:
with_obs["runtime"] = 25
res = round(with_obs.drop(columns="moet").groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]).sort_values(by=["ml_controller"]),2)
#res[res.columns[:-4]]
print(res[res.columns[:-4]].to_latex())

\begin{tabular}{lllrrrr}
\toprule
    &     &    & \multicolumn{2}{l}{mean\_et} & \multicolumn{2}{l}{avg\_iterations} \\
    &     &    &    mean &   std &           mean &   std \\
ml\_controller & speed & runtime &         &       &                &       \\
\midrule
ARS & 0.5 & 25 &   25.71 &  1.05 &          10.01 &  0.37 \\
    & 1.0 & 25 &   24.38 &  4.26 &          11.38 &  1.66 \\
    & 1.5 & 25 &   22.12 &  3.40 &          11.83 &  1.54 \\
E2E (Camera) & 0.5 & 25 &   34.04 &  0.76 &           6.02 &  0.03 \\
    & 1.0 & 25 &   32.63 &  1.43 &           6.77 &  0.70 \\
    & 1.5 & 25 &   32.21 &  1.44 &           6.97 &  0.76 \\
E2E (Lidar) & 0.5 & 25 &   34.45 &  0.53 &           6.02 &  0.01 \\
    & 1.0 & 25 &   34.23 &  2.90 &           6.98 &  1.18 \\
    & 1.5 & 25 &   32.20 &  6.79 &           7.27 &  2.91 \\
SAC & 0.5 & 25 &   26.88 &  0.91 &           9.51 &  0.16 \\
    & 1.0 & 25 &   23.54 &  3.28 &          10.86 &  1.60 \\
    & 1.5 & 25 &   26.16 &  4.24 &        

/home/musaup/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [240]:
with_obs["runtime"] = 25
res = round(with_obs.drop(columns="moet").groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]).sort_values(by=["ml_controller"]),2)
res[res.columns[4:]]
print(res[res.columns[4:]].to_latex())

\begin{tabular}{lllrrrr}
\toprule
    &     &    & \multicolumn{2}{l}{ml\_usage} & \multicolumn{2}{l}{sc\_usage} \\
    &     &    &     mean &    std &     mean &    std \\
ml\_controller & speed & runtime &          &        &          &        \\
\midrule
ARS & 0.5 & 25 &    55.26 &   2.95 &    44.74 &   2.95 \\
    & 1.0 & 25 &    26.39 &   3.81 &    73.61 &   3.81 \\
    & 1.5 & 25 &    14.11 &   2.04 &    85.89 &   2.04 \\
E2E (Camera) & 0.5 & 25 &    84.75 &   2.68 &    15.25 &   2.68 \\
    & 1.0 & 25 &    49.73 &   8.18 &    50.27 &   8.18 \\
    & 1.5 & 25 &    36.04 &  10.78 &    63.96 &  10.78 \\
E2E (Lidar) & 0.5 & 25 &    92.46 &   1.63 &     7.54 &   1.63 \\
    & 1.0 & 25 &    52.30 &   6.35 &    47.70 &   6.35 \\
    & 1.5 & 25 &    25.66 &  10.15 &    74.34 &  10.15 \\
SAC & 0.5 & 25 &    53.29 &   3.66 &    46.71 &   3.66 \\
    & 1.0 & 25 &    18.97 &   3.15 &    81.03 &   3.15 \\
    & 1.5 & 25 &     9.25 &   2.33 &    90.75 &   2.33 \\
\bottomrule
\end{tabular}



/home/musaup/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [241]:
res = round(with_obs.groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]).sort_values(by=["ml_controller"]),2)
res.columns

MultiIndex([(          'moet', 'mean'),
            (          'moet',  'std'),
            (       'mean_et', 'mean'),
            (       'mean_et',  'std'),
            ('avg_iterations', 'mean'),
            ('avg_iterations',  'std'),
            (      'ml_usage', 'mean'),
            (      'ml_usage',  'std'),
            (      'sc_usage', 'mean'),
            (      'sc_usage',  'std')],
           )

In [242]:
no_obs["runtime"] = 25
res = round(no_obs.drop(columns="moet").groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]).sort_values(by=["ml_controller"]),2)

res=res.reset_index()
print(res[res.columns[3:7]].to_latex())

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{2}{l}{mean\_et} & \multicolumn{2}{l}{avg\_iterations} \\
{} &    mean &   std &           mean &   std \\
\midrule
0  &   26.83 &  1.68 &          10.26 &  0.23 \\
1  &   21.61 &  3.61 &          11.63 &  1.63 \\
2  &   20.32 &  3.46 &          12.11 &  1.59 \\
3  &   33.74 &  0.78 &           6.01 &  0.02 \\
4  &   33.36 &  0.57 &           6.18 &  0.17 \\
5  &   30.48 &  2.99 &           7.69 &  1.46 \\
6  &   36.49 &  1.10 &           6.00 &  0.02 \\
7  &   34.65 &  5.13 &           6.50 &  2.14 \\
8  &   36.11 &  5.84 &           6.68 &  2.13 \\
9  &   28.15 &  1.83 &           9.60 &  0.26 \\
10 &   27.32 &  5.54 &           9.01 &  2.52 \\
11 &   25.94 &  4.22 &           9.70 &  1.89 \\
\bottomrule
\end{tabular}



/home/musaup/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [243]:
with_obs["runtime"] = 25
res = round(with_obs.drop(columns="moet").groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]).sort_values(by=["ml_controller"]),2)
res

/home/musaup/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


mean_et       avg_iterations       ml_usage  \
                               mean   std           mean   std     mean   
ml_controller speed runtime                                               
ARS           0.5   25        25.71  1.05          10.01  0.37    55.26   
              1.0   25        24.38  4.26          11.38  1.66    26.39   
              1.5   25        22.12  3.40          11.83  1.54    14.11   
E2E (Camera)  0.5   25        34.04  0.76           6.02  0.03    84.75   
              1.0   25        32.63  1.43           6.77  0.70    49.73   
              1.5   25        32.21  1.44           6.97  0.76    36.04   
E2E (Lidar)   0.5   25        34.45  0.53           6.02  0.01    92.46   
              1.0   25        34.23  2.90           6.98  1.18    52.30   
              1.5   25        32.20  6.79           7.27  2.91    25.66   
SAC           0.5   25        26.88  0.91           9.51  0.16    53.29   
              1.0   25        23.54  3.28          10.86  1.60    18.97   
              1.5   25        26.16  4.24           9.69  1.72     9.25   

                                   sc_usage         
                               std     mean    std  
ml_controller speed runtime                         
ARS           0.5   25        2.95    44.74   2.95  
              1.0   25        3.81    73.61   3.81  
              1.5   25        2.04    85.89   2.04  
E2E (Camera)  0.5   25        2.68    15.25   2.68  
              1.0   25        8.18    50.27   8.18  
              1.5   25       10.78    63.96  10.78  
E2E (Lidar)   0.5   25        1.63     7.54   1.63  
              1.0   25        6.35    47.70   6.35  
              1.5   25       10.15    74.34  10.15  
SAC           0.5   25        3.66    46.71   3.66  
              1.0   25        3.15    81.03   3.15  
              1.5   25        2.33    90.75   2.33

In [244]:
no_obs["runtime"] = 25
res = round(no_obs.drop(columns="moet").groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]).sort_values(by=["ml_controller"]),2)
res[res.columns[4:]]
print(res[res.columns[4:]].to_latex())

/home/musaup/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


\begin{tabular}{lllrrrr}
\toprule
    &     &    & \multicolumn{2}{l}{ml\_usage} & \multicolumn{2}{l}{sc\_usage} \\
    &     &    &     mean &    std &     mean &    std \\
ml\_controller & speed & runtime &          &        &          &        \\
\midrule
ARS & 0.5 & 25 &    58.86 &   2.58 &    41.14 &   2.58 \\
    & 1.0 & 25 &    32.92 &   7.15 &    67.08 &   7.15 \\
    & 1.5 & 25 &    16.64 &   2.54 &    83.36 &   2.54 \\
E2E (Camera) & 0.5 & 25 &    94.30 &   0.36 &     5.70 &   0.36 \\
    & 1.0 & 25 &    73.25 &  12.11 &    26.75 &  12.11 \\
    & 1.5 & 25 &    40.10 &  13.35 &    59.90 &  13.35 \\
E2E (Lidar) & 0.5 & 25 &   100.00 &   0.00 &     0.00 &   0.00 \\
    & 1.0 & 25 &    89.40 &  18.34 &    10.60 &  18.34 \\
    & 1.5 & 25 &    26.61 &  17.68 &    73.39 &  17.68 \\
SAC & 0.5 & 25 &    55.62 &   6.37 &    44.38 &   6.37 \\
    & 1.0 & 25 &    16.98 &  10.25 &    83.02 &  10.25 \\
    & 1.5 & 25 &    10.27 &   3.58 &    89.73 &   3.58 \\
\bottomrule
\end{tabular}



In [245]:
no_obs["runtime"] = 25
res = round(no_obs.drop(columns="moet").groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]).sort_values(by=["ml_controller"]),2)
res[res.columns[4:]]

/home/musaup/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


ml_usage        sc_usage       
                                mean    std     mean    std
ml_controller speed runtime                                
ARS           0.5   25         58.86   2.58    41.14   2.58
              1.0   25         32.92   7.15    67.08   7.15
              1.5   25         16.64   2.54    83.36   2.54
E2E (Camera)  0.5   25         94.30   0.36     5.70   0.36
              1.0   25         73.25  12.11    26.75  12.11
              1.5   25         40.10  13.35    59.90  13.35
E2E (Lidar)   0.5   25        100.00   0.00     0.00   0.00
              1.0   25         89.40  18.34    10.60  18.34
              1.5   25         26.61  17.68    73.39  17.68
SAC           0.5   25         55.62   6.37    44.38   6.37
              1.0   25         16.98  10.25    83.02  10.25
              1.5   25         10.27   3.58    89.73   3.58

In [246]:
with_obs["runtime"] = 25
res = round(with_obs.drop(columns="moet").groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]).sort_values(by=["ml_controller"]),2)
res[res.columns[4:-1]]

/home/musaup/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


ml_usage        sc_usage
                                mean    std     mean
ml_controller speed runtime                         
ARS           0.5   25         55.26   2.95    44.74
              1.0   25         26.39   3.81    73.61
              1.5   25         14.11   2.04    85.89
E2E (Camera)  0.5   25         84.75   2.68    15.25
              1.0   25         49.73   8.18    50.27
              1.5   25         36.04  10.78    63.96
E2E (Lidar)   0.5   25         92.46   1.63     7.54
              1.0   25         52.30   6.35    47.70
              1.5   25         25.66  10.15    74.34
SAC           0.5   25         53.29   3.66    46.71
              1.0   25         18.97   3.15    81.03
              1.5   25          9.25   2.33    90.75

### 10 ms Experiments

In [247]:
cols={0:"time_string",1:"time_taken_lec",2:"time_taken_safety_controller",3:"total_time",4:"moet",5:"mean_et",6:"avg_iterations"}
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0","1.5"]
file_str = "10ms/benchmark_experiments_{}_track_porto_{}.csv"
dfs2 = []
for alg in algorithms:
    for speed in speeds:
            filepath = file_str.format(alg,speed)
            try:
                df = df = pd.read_csv(filepath,header=None).rename(columns=cols).tail(30)
            except:
                print(filepath)
                continue
            df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
            df=df.drop(columns=["time_taken_lec","total_time","time_taken_safety_controller"])
            df["sc_usage"] = 1 - df["ml_usage"]
            df["ml_usage"]=df["ml_usage"]*100
            df["sc_usage"]=df["sc_usage"]*100
            df["ml_controller"] = alg.replace("_rtreach","")
            df["speed"] = speed
            df["obstacle_presence"] = obs_presence
            df["runtime"] = 10
            dfs2.append(df)

10ms/benchmark_experiments_e2e_rtreach_track_porto_1.0.csv
10ms/benchmark_experiments_sac_rtreach_track_porto_1.0.csv
10ms/benchmark_experiments_ars_rtreach_track_porto_1.0.csv
10ms/benchmark_experiments_ars_rtreach_track_porto_1.5.csv


In [248]:
all_dfs2 = pd.concat(dfs2)
all_dfs2["ml_controller"]=all_dfs2["ml_controller"].apply(replace_e2e)
all_dfs2["obstacle_presence"] = all_dfs2["obstacle_presence"].apply(replace_obs)
all_dfs2.groupby(["ml_controller","obstacle_presence","speed"]).count()

time_string  moet  mean_et  \
ml_controller obstacle_presence speed                               
ARS           Yes               0.5             30    30       30   
E2E (Camera)  Yes               0.5             30    30       30   
                                1.0             30    30       30   
                                1.5             30    30       30   
E2E (Lidar)   Yes               0.5             30    30       30   
                                1.5             30    30       30   
SAC           Yes               0.5             30    30       30   
                                1.5             14    14       14   

                                       avg_iterations  ml_usage  sc_usage  \
ml_controller obstacle_presence speed                                       
ARS           Yes               0.5                30        30        30   
E2E (Camera)  Yes               0.5                30        30        30   
                                1.0                30        30        30   
                                1.5                30        30        30   
E2E (Lidar)   Yes               0.5                30        30        30   
                                1.5                30        30        30   
SAC           Yes               0.5                30        30        30   
                                1.5                14        14        14   

                                       runtime  
ml_controller obstacle_presence speed           
ARS           Yes               0.5         30  
E2E (Camera)  Yes               0.5         30  
                                1.0         30  
                                1.5         30  
E2E (Lidar)   Yes               0.5         30  
                                1.5         30  
SAC           Yes               0.5         30  
                                1.5         14

In [249]:
pd.concat([with_obs

SyntaxError: unexpected EOF while parsing (<ipython-input-249-d7493097dffc>, line 1)

In [250]:
res2 = round(all_dfs2.drop(columns="moet").groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]).sort_values(by=["ml_controller"]),2)
res2

mean_et       avg_iterations       ml_usage  \
                               mean   std           mean   std     mean   
ml_controller speed runtime                                               
ARS           0.5   10         0.01  0.00           9.13  0.34    55.19   
E2E (Camera)  0.5   10         0.02  0.00           4.99  0.01    84.89   
              1.0   10         0.01  0.00           6.09  1.05    46.96   
              1.5   10         0.02  0.00           6.01  0.72    35.65   
E2E (Lidar)   0.5   10         0.02  0.00           4.99  0.01    92.22   
              1.5   10         0.01  0.01           8.66  5.01    23.18   
SAC           0.5   10         0.01  0.00           8.56  0.20    53.66   
              1.5   10         0.01  0.00          10.28  0.99     9.80   

                                   sc_usage         
                               std     mean    std  
ml_controller speed runtime                         
ARS           0.5   10        2.43    44.81   2.43  
E2E (Camera)  0.5   10        2.52    15.11   2.52  
              1.0   10       10.96    53.04  10.96  
              1.5   10        8.01    64.35   8.01  
E2E (Lidar)   0.5   10        2.20     7.78   2.20  
              1.5   10       12.23    76.82  12.23  
SAC           0.5   10        4.35    46.34   4.35  
              1.5   10        2.27    90.20   2.27

In [251]:
ls 10ms

benchmark_experiments_ars_rtreach_track_porto_0.5.csv
benchmark_experiments_e2e_image_rtreach_track_porto_0.5.csv
benchmark_experiments_e2e_image_rtreach_track_porto_1.0.csv
benchmark_experiments_e2e_image_rtreach_track_porto_1.5.csv
benchmark_experiments_e2e_rtreach_track_porto_0.5.csv
benchmark_experiments_e2e_rtreach_track_porto_1.5.csv
benchmark_experiments_sac_rtreach_track_porto_0.5.csv
benchmark_experiments_sac_rtreach_track_porto_1.5.csv
